# Building an E-Model for a single-compartment cell

This notebook provides a simple example on how to run the pipeline to create an e-model of a single-compartment cell with two free parameters. In this instance, we will use non-threshold optimisation. For threshold-based optimization or additional details, please refer to the [L5PC](./../L5PC/) example.

In [1]:
import json
from IPython.display import display, JSON

from bluepyemodel.emodel_pipeline.emodel_pipeline import EModel_pipeline
from bluepyemodel.efeatures_extraction.targets_configurator import TargetsConfigurator

In [ ]:
# Clear any existing checkpoints to avoid conflicts with previous runs
!rm -r ./checkpoints

## Setting up the pipeline

The [recipes.json](./config/recipes.json) file (displayed below) contains the key settings for the various stages of the e-model building pipeline.

Since we aim to perform non-threshold optimization, we set ``extract_absolute_amplitudes`` to ``true`` in the ``pipeline_settings``

In [3]:
recipes_path = "./config/recipes.json"
with open(recipes_path, 'r') as file:
    recipe = json.load(file)

print(json.dumps(recipe, indent=4))

{
    "simplecell": {
        "morph_path": "./morphologies/",
        "morphology": [
            [
                "simple",
                "simple.swc"
            ]
        ],
        "params": "config/params/simple.json",
        "features": "config/features/simplecell.json",
        "pipeline_settings": {
            "path_extract_config": "config/extract_config/simplecell_config.json",
            "plot_extraction": true,
            "default_std_value": 0.01,
            "extract_absolute_amplitudes": true,
            "efel_settings": {
                "strict_stiminterval": true,
                "Threshold": -20.0,
                "interp_step": 0.025
            },
            "optimisation_timeout": 300,
            "optimiser": "SO-CMA",
            "max_ngen": 5,
            "optimisation_params": {
                "offspring_size": 20
            },
            "validation_threshold": 100,
            "plot_currentscape": true,
            "currentscape_config": {
     

First, we need to create the `pipeline` using the `EModel_pipeline` class. This class loads the `recipe.json` file and sets up the pipeline accordingly. Additionally, we need to provide some metadata, including the e-model name, e-type, brain region, and morphology.

In [4]:
emodel = "simplecell"
etype = "cAC"
species = "mouse"
brain_region = "SSCX"
morphology = "simple"
morphology_format = "swc"

In [5]:
pipeline = EModel_pipeline(
    emodel=emodel,
    etype=etype,
    species=species,
    brain_region=brain_region,
    recipes_path=recipes_path,
    data_access_point="local",
)

## Extracting the features

The first stage of our pipeline is to extract the relevant features. In this step, we define the electrophysiological data that will be used to fit the model. The data 

First, we need to download the necessary data using the script [download_ephys_data.sh](./download_ephys_data.sh).

In [ ]:
!sh ./download_ephys_data.sh

In [7]:
filenames = [
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_326.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_327.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_328.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_329.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_330.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_349.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_350.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_351.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_352.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_353.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_266.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_267.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_268.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_269.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_270.ibw"
]

The we define the targets which contains the protocols (ecodes) and features. The extracted features guide the optimisation of the model to match experimental data at specified amplitudes. For the IDrest protocol with an amplitude of 0.256 nA, features like Spikecount, mean_frequency, and voltage_base capture the neuron's spiking activity and resting potential. In the IV protocol, at an amplitude of -0.147 nA, voltage_base and ohmic_input_resistance_vb_ssse assess the neuron's baseline potential and input resistance. 

In [8]:
targets = {
    "IDrest": {
        "amplitudes": [0.256],
        "efeatures": [
            "Spikecount",
            "mean_frequency",
            "voltage_base",
        ],
    },
    "IV": {
        "amplitudes": [-0.147],
        "efeatures": [
            "voltage_base",
            "ohmic_input_resistance_vb_ssse",
        ],
    }
}

The ecodes_metadata dictionary defines parameters for each protocol: ljp is the liquid junction potential correction (14.0 mV), while ton and toff represent the start and stop times (in ms) for current injection.

In [9]:
ecodes_metadata = {
    "IDthresh": {"ljp": 14.0, "ton": 700, "toff": 2700},
    "IDrest": {"ljp": 14.0, "ton": 700, "toff": 2700},
    "IV": {"ljp": 14.0, "ton": 20, "toff": 1020},
}

We save this targets in an object called `ExtractionTargetConfigurator` (ETC), shown below, which will serve as the input for the feature extractor

In [10]:
files_metadata = []
for filename in filenames:
    fn = filename.split("/")[-1]
    for ecode in ecodes_metadata:
        if ecode in fn:
            files_metadata.append(
                {
                    "cell_name": filename.split("/")[-2],
                    "filename": filename.split("/")[-1].split(".")[0],
                    "ecodes": {ecode: ecodes_metadata[ecode]},
                    "other_metadata": {
                        "i_file": filename,
                        "v_file": filename.replace("ch0", "ch1"),
                        "i_unit": "A",
                        "v_unit": "V",
                        "t_unit": "s",
                    },
                }
            )


targets_formated = []
for ecode in targets:
    for amplitude in targets[ecode]["amplitudes"]:
        for efeature in targets[ecode]["efeatures"]:
            targets_formated.append(
                {
                    "efeature": efeature,
                    "protocol": ecode,
                    "amplitude": amplitude,
                    "tolerance": 0.1,
                }
            )


configurator = TargetsConfigurator(pipeline.access_point)
configurator.new_configuration(files_metadata, targets_formated)
configurator.save_configuration()
print(json.dumps(configurator.configuration.as_dict(), indent=4))

{
    "files": [
        {
            "cell_name": "C060109A1-SR-C1",
            "filename": "X_IDrest_ch0_326",
            "filepath": null,
            "resource_id": null,
            "ecodes": {
                "IDrest": {
                    "ljp": 14.0,
                    "ton": 700,
                    "toff": 2700
                }
            },
            "other_metadata": {
                "i_file": "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_326.ibw",
                "v_file": "./ephys_data/C060109A1-SR-C1/X_IDrest_ch1_326.ibw",
                "i_unit": "A",
                "v_unit": "V",
                "t_unit": "s"
            },
            "species": null,
            "brain_region": null,
            "etype": null,
            "id": null
        },
        {
            "cell_name": "C060109A1-SR-C1",
            "filename": "X_IDrest_ch0_327",
            "filepath": null,
            "resource_id": null,
            "ecodes": {
                "IDrest": {
     

We can now proceed to extract the e-features

In [ ]:
pipeline.extract_efeatures()

The results of the feature extraction is stored in [simplecell.json](./config/features/simplecell.json). Let's take a look at the extracted features

In [12]:
fcc_path = "./config/features/simplecell.json"
with open(fcc_path, 'r') as file:
    fcc = json.load(file)

print(json.dumps(fcc, indent=4))

{
    "efeatures": [
        {
            "efel_feature_name": "Spikecount",
            "protocol_name": "IDrest_0.256",
            "recording_name": "soma.v",
            "threshold_efeature_std": null,
            "default_std_value": 0.01,
            "mean": 6.6,
            "original_std": 4.5431266766402185,
            "sample_size": 5,
            "efeature_name": "Spikecount",
            "weight": 1.0,
            "efel_settings": {
                "strict_stiminterval": true,
                "Threshold": -20.0,
                "interp_step": 0.025
            }
        },
        {
            "efel_feature_name": "mean_frequency",
            "protocol_name": "IDrest_0.256",
            "recording_name": "soma.v",
            "threshold_efeature_std": null,
            "default_std_value": 0.01,
            "mean": 4.6549665308108965,
            "original_std": 1.7066939148229534,
            "sample_size": 4,
            "efeature_name": "mean_frequency",
            "

## Setting the parameter for the optimisation

The parameters for the optimisation are defined in [simple.json](./config/simple.json). In this case, we are optimizing two parameters: gnabar_hh and gkbar_hh. These parameters determine the maximum conductances for sodium and potassium ion channels in a Hodgkin-Huxley neuron model, with optimization ranges of 0.05–0.125 and 0.01–0.075, respectively.

In [13]:
# EMC
emc_path = "./config/params/simple.json"
with open(emc_path, 'r') as file:
    emc = json.load(file)

print(json.dumps(emc, indent=4))

{
    "mechanisms": {
        "somatic": {
            "mech": [
                "hh"
            ]
        }
    },
    "distributions": {},
    "parameters": {
        "__comment": "define constants as single values and params to optimise as tuples of bounds: [lower, upper]",
        "global": [
            {
                "name": "v_init",
                "val": -80
            },
            {
                "name": "celsius",
                "val": 34
            }
        ],
        "somatic": [
            {
                "name": "Ra",
                "val": 100
            },
            {
                "name": "cm",
                "val": 1
            },
            {
                "name": "ena",
                "val": 50
            },
            {
                "name": "ek",
                "val": -90
            },
            {
                "name": "gnabar_hh",
                "val": [
                    0.05,
                    0.125
                ]
  

## Running the optimisation

We can now run the optimisation using the `EModel_pipeline` class. The optimisation will run for 5 generations, with a population size of 20.

In [ ]:
pipeline.optimise(seed=1)

Once the model has been fitted, we can store the results in the [final.json](./final.json) file.

In [ ]:
pipeline.store_optimisation_results()

## Validating the model

We can also run the validation of the e-model, executing the protocols defined under the `validation_protocols` key in  recipes.json

In [ ]:
pipeline.validation()

## Plotting the results

Finally, we can generate various plots, including the resulting traces, optimisation plots, and scores. These plots will be saved in the `./figures` directory.

In [ ]:
pipeline.plot(only_validated=False)